In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    openssl-1.1.1j             |       h7f98852_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py

In [5]:
neighborhoods= pd.read_csv("./Linnaosad.csv")
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Tallinn,Haabersti,59.431328,24.629042
1,Tallinn,Kesklinn,59.436960,24.753530
2,Tallinn,Kristiine,59.415000,24.709150
3,Tallinn,Lasnamäe,59.433600,24.860170
4,Tallinn,Mustamäe,59.404820,24.681190


In [6]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 1 boroughs and 8 neighborhoods.


In [7]:
address = 'Tallinn, Estonia'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tallinn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tallinn are 59.4372155, 24.7453688.


In [8]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [9]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Tallinn'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Tallinn,Haabersti,59.431328,24.629042
1,Tallinn,Kesklinn,59.436960,24.753530
2,Tallinn,Kristiine,59.415000,24.709150
3,Tallinn,Lasnamäe,59.433600,24.860170
4,Tallinn,Mustamäe,59.404820,24.681190


In [10]:
address = 'Tallinn, estonia'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tallinn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tallinn are 59.4372155, 24.7453688.


In [14]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [15]:
CLIENT_ID = 'A42H3MCRDBJ0UJ0LQREYLJNKB2R1AQOM0FT1NUHWY5AGLVF5' # your Foursquare ID
CLIENT_SECRET = 'PKVYIP2SRSATJBW0RGM1Z0CEZ1VZFSJ5I5EUOMQ23DJCZHXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 999 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: A42H3MCRDBJ0UJ0LQREYLJNKB2R1AQOM0FT1NUHWY5AGLVF5
CLIENT_SECRET:PKVYIP2SRSATJBW0RGM1Z0CEZ1VZFSJ5I5EUOMQ23DJCZHXX


In [16]:
manhattan_data.loc[0, 'Neighborhood']

'Haabersti'

In [17]:
manhattan_data.count

<bound method DataFrame.count of    Borough   Neighborhood   Latitude  Longitude
0  Tallinn      Haabersti  59.431328  24.629042
1  Tallinn       Kesklinn  59.436960  24.753530
2  Tallinn      Kristiine  59.415000  24.709150
3  Tallinn       Lasnamäe  59.433600  24.860170
4  Tallinn       Mustamäe  59.404820  24.681190
5  Tallinn          Nõmme  59.377570  24.685870
6  Tallinn         Pirita  59.474869  24.872448
7  Tallinn  Põhja-Tallinn  59.458199  24.683699>

In [18]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Haabersti are 59.4313282, 24.6290422.


In [19]:
#The correct answer is:
LIMIT = 999 # limit of number of venues returned by Foursquare API
radius = 55000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=A42H3MCRDBJ0UJ0LQREYLJNKB2R1AQOM0FT1NUHWY5AGLVF5&client_secret=PKVYIP2SRSATJBW0RGM1Z0CEZ1VZFSJ5I5EUOMQ23DJCZHXX&v=20180605&ll=59.4313282,24.6290422&radius=55000&limit=999'

In [94]:
results = requests.get(url).json()
results
results['response'] ['totalResults']

174

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [40]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,MyFitness Rocca al Mare,Gym / Fitness Center,59.426810,24.645440
1,Rocca Al Mare promenaad,Trail,59.428971,24.654436
2,Eesti Vabaõhumuuseum,Museum,59.433537,24.640577
3,Nõmme-Harku Terviserada,Trail,59.387124,24.652119
4,Rucola Ristorante & Pizzeria,Italian Restaurant,59.414333,24.660145


In [41]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [144]:
nearby_venues


,name,categories,lat,lng
0,MyFitness Rocca al Mare,Gym / Fitness Center,59.426810,24.645440
1,Rocca Al Mare promenaad,Trail,59.428971,24.654436
2,Eesti Vabaõhumuuseum,Museum,59.433537,24.640577
3,Nõmme-Harku Terviserada,Trail,59.387124,24.652119
4,Rucola Ristorante & Pizzeria,Italian Restaurant,59.414333,24.660145
5,Bauhaus,Hardware Store,59.425561,24.640486
6,Kohtuotsa vaateplatvorm (Kohtuotsa Vaateplatvorm),Scenic Lookout,59.437747,24.742215
7,Rataskaevu 16,Eastern European Restaurant,59.436808,24.742454
8,Kohvik Grenka,Café,59.423874,24.742358
9,Uulits,Burger Joint,59.411994,24.667994


In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:
nearby_venues

,name,categories,lat,lng
0,MyFitness Rocca al Mare,Gym / Fitness Center,59.426810,24.645440
1,Rocca Al Mare promenaad,Trail,59.428971,24.654436
2,Eesti Vabaõhumuuseum,Museum,59.433537,24.640577
3,Nõmme-Harku Terviserada,Trail,59.387124,24.652119
4,Rucola Ristorante & Pizzeria,Italian Restaurant,59.414333,24.660145
5,Bauhaus,Hardware Store,59.425561,24.640486
6,Kohtuotsa vaateplatvorm (Kohtuotsa Vaateplatvorm),Scenic Lookout,59.437747,24.742215
7,Rataskaevu 16,Eastern European Restaurant,59.436808,24.742454
8,Kohvik Grenka,Café,59.423874,24.742358
9,Uulits,Burger Joint,59.411994,24.667994


In [56]:
nearby_venues.to_csv('nearby_venues.csv') 

In [44]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Haabersti
Kesklinn
Kristiine
Lasnamäe
Mustamäe
Nõmme
Pirita
Põhja-Tallinn


In [47]:
print(manhattan_venues.shape)
manhattan_venues.head()

(135, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Haabersti,59.431328,24.629042,Pet City,59.428567,24.625741,Pet Store
1,Haabersti,59.431328,24.629042,Selver,59.428252,24.626269,Supermarket
2,Haabersti,59.431328,24.629042,Pikaliiva,59.428133,24.627824,Bus Stop
3,Haabersti,59.431328,24.629042,Kakumäe Selveri Südameapteek,59.428191,24.626174,Pharmacy
4,Haabersti,59.431328,24.629042,Räga,59.428237,24.625291,Bus Station


In [48]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Haabersti,7,7,7,7,7,7
Kesklinn,92,92,92,92,92,92
Kristiine,10,10,10,10,10,10
Lasnamäe,5,5,5,5,5,5
Mustamäe,8,8,8,8,8,8
Nõmme,4,4,4,4,4,4
Pirita,3,3,3,3,3,3
Põhja-Tallinn,6,6,6,6,6,6


In [99]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 76 uniques categories.


In [49]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Asian Restaurant,Athletics & Sports,Auto Garage,Bar,Basketball Court,Beer Bar,Bistro,Bookstore,Boutique,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Candy Store,Cheese Shop,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Flower Shop,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Light Rail Station,Liquor Store,Lounge,Mexican Restaurant,Mobile Phone Shop,Movie Theater,Multiplex,Nightclub,Opera House,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pub,Restaurant,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Theater,Wine Bar
0,Haabersti,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Haabersti,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,Haabersti,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Haabersti,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Haabersti,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [50]:
manhattan_onehot.shape

(135, 78)

In [52]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Asian Restaurant,Athletics & Sports,Auto Garage,Bar,Basketball Court,Beer Bar,Bistro,Bookstore,Boutique,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Candy Store,Cheese Shop,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Flower Shop,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health Food Store,Historic Site,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Light Rail Station,Liquor Store,Lounge,Mexican Restaurant,Mobile Phone Shop,Movie Theater,Multiplex,Nightclub,Opera House,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pub,Restaurant,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Theater,Wine Bar
0,Haabersti,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.00000,0.000000,0.00000,0.142857,0.142857,0.142857,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.000000,0.142857,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.0,0.00000,0.000,0.0,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.142857,0.142857,0.00000,0.00,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.142857,0.00000,0.0,0.0,0.00000,0.00000
1,Kesklinn,0.032609,0.000000,0.0,0.043478,0.000000,0.01087,0.01087,0.021739,0.01087,0.000000,0.000000,0.000000,0.01087,0.043478,0.01087,0.01087,0.054348,0.01087,0.021739,0.000000,0.021739,0.01087,0.01087,0.01087,0.01087,0.01087,0.021739,0.0,0.01087,0.000,0.0,0.01087,0.01087,0.01087,0.01087,0.000000,0.01087,0.01087,0.043478,0.01087,0.01087,0.01087,0.076087,0.01087,0.01087,0.01087,0.01087,0.054348,0.01087,0.000000,0.010870,0.01087,0.01087,0.000,0.01087,0.01087,0.01087,0.01087,0.010870,0.01087,0.000000,0.000000,0.01087,0.00,0.0,0.021739,0.021739,0.032609,0.01087,0.01087,0.01087,0.021739,0.01087,0.0,0.0,0.01087,0.01087
2,Kristiine,0.000000,0.000000,0.1,0.000000,0.000000,0.00000,0.10000,0.000000,0.00000,0.100000,0.100000,0.000000,0.00000,0.100000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.0,0.00000,0.000,0.1,0.10000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.10,0.1,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.0,0.1,0.00000,0.00000
3,Lasnamäe,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.200000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.200000,0.2,0.00000,0.000,0.0,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.200000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00,0.0,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.2,0.0,0.00000,0.00000
4,Mustamäe,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.00000,0.000000,0.00000,0.125000,0.000000,0.125000,0.00000,0.000000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.0,0.00000,0.125,0.0,0.00000,0.00000,0.00000,0.00000,0.000000,0.12500,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.125,0.00000,

In [53]:
manhattan_grouped.shape

(8, 78)

In [54]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Haabersti----
               venue  freq
0           Pharmacy  0.14
1        Supermarket  0.14
2          Pet Store  0.14
3  Convenience Store  0.14
4        Bus Station  0.14


----Kesklinn----
                  venue  freq
0                 Hotel  0.08
1    Italian Restaurant  0.05
2          Cocktail Bar  0.05
3  Gym / Fitness Center  0.04
4                  Café  0.04


----Kristiine----
                  venue  freq
0           Auto Garage   0.1
1       Thai Restaurant   0.1
2                Bistro   0.1
3  Fast Food Restaurant   0.1
4              Bus Line   0.1


----Lasnamäe----
                         venue  freq
0         Gym / Fitness Center   0.2
1                 Tennis Court   0.2
2  Eastern European Restaurant   0.2
3            Electronics Store   0.2
4                         Café   0.2


----Mustamäe----
               venue  freq
0        Pizza Place  0.25
1           Bus Stop  0.12
2      Grocery Store  0.12
3  Mobile Phone Shop  0.12
4           Bus Line  0.12

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [95]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Haabersti,Bus Line,Bus Stop,Supermarket,Pet Store,Pharmacy,Convenience Store,Bus Station,Cupcake Shop,Dance Studio,Department Store
1,Kesklinn,Hotel,Cocktail Bar,Italian Restaurant,Gym / Fitness Center,Bar,Café,Restaurant,Asian Restaurant,Cosmetics Shop,Pub
2,Kristiine,Playground,Thai Restaurant,Auto Garage,Flower Shop,Bistro,Fast Food Restaurant,Bus Line,Pizza Place,Bus Station,Café
3,Lasnamäe,Gym / Fitness Center,Electronics Store,Tennis Court,Café,Eastern European Restaurant,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dumpling Restaurant
4,Mustamäe,Pizza Place,Bus Line,Grocery Store,Farmers Market,Bus Stop,Pub,Mobile Phone Shop,Cupcake Shop,Dance Studio,Department Store


In [57]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 4, 0, 3, 2, 1], dtype=int32)

In [58]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Tallinn,Haabersti,59.431328,24.629042,0,Bus Line,Bus Stop,Supermarket,Pet Store,Pharmacy,Convenience Store,Bus Station,Cupcake Shop,Dance Studio,Department Store
1,Tallinn,Kesklinn,59.436960,24.753530,0,Hotel,Cocktail Bar,Italian Restaurant,Gym / Fitness Center,Bar,Café,Restaurant,Asian Restaurant,Cosmetics Shop,Pub
2,Tallinn,Kristiine,59.415000,24.709150,0,Playground,Thai Restaurant,Auto Garage,Flower Shop,Bistro,Fast Food Restaurant,Bus Line,Pizza Place,Bus Station,Café
3,Tallinn,Lasnamäe,59.433600,24.860170,4,Gym / Fitness Center,Electronics Store,Tennis Court,Café,Eastern European Restaurant,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dumpling Restaurant
4,Tallinn,Mustamäe,59.404820,24.681190,0,Pizza Place,Bus Line,Grocery Store,Farmers Market,Bus Stop,Pub,Mobile Phone Shop,Cupcake Shop,Dance Studio,Department Store


In [59]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [60]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Haabersti,Bus Line,Bus Stop,Supermarket,Pet Store,Pharmacy,Convenience Store,Bus Station,Cupcake Shop,Dance Studio,Department Store
1,Kesklinn,Hotel,Cocktail Bar,Italian Restaurant,Gym / Fitness Center,Bar,Café,Restaurant,Asian Restaurant,Cosmetics Shop,Pub
2,Kristiine,Playground,Thai Restaurant,Auto Garage,Flower Shop,Bistro,Fast Food Restaurant,Bus Line,Pizza Place,Bus Station,Café
4,Mustamäe,Pizza Place,Bus Line,Grocery Store,Farmers Market,Bus Stop,Pub,Mobile Phone Shop,Cupcake Shop,Dance Studio,Department Store


In [61]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Põhja-Tallinn,Bus Station,Basketball Court,Liquor Store,Light Rail Station,Bus Stop,Wine Bar,Dance Studio,Department Store,Dessert Shop,Dumpling Restaurant


In [62]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Pirita,Athletics & Sports,Garden,Park,Wine Bar,Electronics Store,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dumpling Restaurant


In [63]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Nõmme,Park,Bus Stop,Restaurant,Wine Bar,Eastern European Restaurant,Cosmetics Shop,Cupcake Shop,Dance Studio,Department Store,Dessert Shop


In [64]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Lasnamäe,Gym / Fitness Center,Electronics Store,Tennis Court,Café,Eastern European Restaurant,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dumpling Restaurant


In [65]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 5, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [66]:
df_incidents= pd.read_csv("./kokku.csv")
df_incidents.shape

(617, 4)

In [67]:
df_incidents = df_incidents.iloc[0:999, :]

In [68]:
# The geograpical coordinate of Tallinn are 59.4372155, 24.7453688.
latitude = 59.4372155
longitude = 24.7453688


In [69]:
# create map and display it
sanfran_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# display the map of San Francisco
sanfran_map

In [70]:
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(df_incidents.lat, df_incidents.lng):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
sanfran_map.add_child(incidents)

In [72]:
# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(df_incidents.lat, df_incidents.lng):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add pop-up text to each marker on the map
latitudes = list(df_incidents.lat)
longitudes = list(df_incidents.lng)
labels = list(df_incidents.categories)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(sanfran_map)    
    
# add incidents to map
sanfran_map.add_child(incidents)

In [74]:
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
sanfran_map = folium.Map(location = [latitude, longitude], zoom_start = 12)

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(sanfran_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(df_incidents.lat, df_incidents.lng, df_incidents.categories):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

# display map
sanfran_map

In [53]:
NearbyVenues= pd.read_csv("./kokku.csv")

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

KeyError: 'Neighborhood'

In [35]:
print(manhattan_venues.shape)
manhattan_venues.head()

(135, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Haabersti,59.431328,24.629042,Pet City,59.428567,24.625741,Pet Store
1,Haabersti,59.431328,24.629042,Selver,59.428252,24.626269,Supermarket
2,Haabersti,59.431328,24.629042,Pikaliiva,59.428133,24.627824,Bus Stop
3,Haabersti,59.431328,24.629042,Kakumäe Selveri Südameapteek,59.428191,24.626174,Pharmacy
4,Haabersti,59.431328,24.629042,Räga,59.428237,24.625291,Bus Station


In [36]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Haabersti,8,8,8,8,8,8
Kesklinn,92,92,92,92,92,92
Kristiine,9,9,9,9,9,9
Lasnamäe,5,5,5,5,5,5
Mustamäe,8,8,8,8,8,8
Nõmme,4,4,4,4,4,4
Pirita,3,3,3,3,3,3
Põhja-Tallinn,6,6,6,6,6,6
